##### Copyright 2025 Google LLC.

In [24]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Basic Code Review

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/prompting/Basic_Code_Review.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/>

Developed by [Shipra](https://github.com/Shi-pra-19).

Gemini API can be a great tool to save you time during the development process by assisting with tasks such as code review, debugging, and optimization.

Gemini API’s Python SDK can be used to perform various forms of code review tasks, including:
- Providing feedback on code quality, readability, or best practices
- Identifying weak areas in the code such as poor variable naming, inefficient loops, or security vulnerabilities
- Suggesting optimizations to improve performance and maintainability


Below is an example of using the Gemini model to enhance code quality through feedback.

In [25]:
%pip install -U -q "google-genai>=1.7.0"

In [26]:
from google import genai

## Configure your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [27]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

client = genai.Client(api_key=GOOGLE_API_KEY)

## Example

Start by defining some system instructions for this problem. For demonstration purposes, this use case involves prompting the model to generate a piece of code that contains a mix of common and critical mistakes.

In [28]:
MODEL_ID="gemini-2.0-flash" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-pro-exp-03-25"] {"allow-input":true, isTemplate: true}

In [29]:
from google.genai import types
from IPython.display import Markdown

buggy_code_prompt = """
     You are a beginner programmer.
     You are tasked with writing code that include common errors:

     1. Logical flaws
     2. Inefficient looping constructs
     3. Inconsistent formatting.
     4. Poor naming conventions

     Only output the code—no explanations.
     If the task doesn’t specify a programming language, default to Python.
"""

programmer_response = client.models.generate_content(
    model=MODEL_ID,
    contents= """
            Write code to take a list of integers, remove any duplicates,
            filter out all numbers less than 30,
            and return the remaining numbers sorted in descending order.
     """,
    config=types.GenerateContentConfig(
        system_instruction=buggy_code_prompt
    ),
)

code = programmer_response.text
Markdown(code)

```python
 def process_numbers(lst):
  new_list = []
  for num in lst:
   if num not in new_list:
    new_list.append(num)
  
  filtered_list = []
  for i in new_list:
   if i >= 30:
    filtered_list.append(i)
  
  filtered_list.sort(reverse=True)
  return filtered_list
```

In [23]:
review_prompt = """
     As a senior programmer, you are tasked with reviewing beginner-level code.
     Please follow these instructions for the evaluation:

     1. Provide detailed feedback on the code, focusing on:
        - Logic errors
        - Code readability and naming conventions
        - Code efficiency and best practices

     2. Write a corrected version of the code, addressing the identified issues.

     3. Clearly point out the changes you made and explain why they improve the original code.
"""

review = client.models.generate_content(
    model=MODEL_ID,
    contents=code,
    config=types.GenerateContentConfig(
        system_instruction=review_prompt
    ),
)


Markdown(review.text)

Okay, I've reviewed the code and here's my feedback.

**1. Feedback:**

*   **Logic Errors:**  The code aims to filter a list of numbers, remove duplicates, keep only numbers greater than or equal to 30, sort the remaining numbers, and return them in descending order. The logic is sound overall, but there's room for improvement in efficiency and clarity.
*   **Code Readability and Naming Conventions:**
    *   Variable names are generally okay (`list_of_nums`, `temp_list`, `new_list`), but could be more descriptive to improve readability. For example, `unique_numbers` instead of `temp_list` would be clearer.
    *   The code is somewhat verbose.  It could be more concise by using list comprehensions and built-in functions.
    *   There are no comments explaining the purpose of each section of the code.
*   **Code Efficiency and Best Practices:**
    *   Using `n not in temp_list` inside a loop has a time complexity of O(n), where n is the length of `temp_list`.  This makes the duplicate removal process inefficient, especially for large lists.  Using a `set` is a much more efficient way to remove duplicates (O(1) average case for checking membership).
    *   Creating multiple lists (`temp_list`, `new_list`) can be avoided by using list comprehensions or generator expressions, making the code more memory-efficient.
    *   Sorting and reversing can be done in a single step using the `sort` method with the `reverse` parameter.

**2. Corrected Version of the Code:**

```python
def process_numbers(numbers):
  """
  Filters a list of numbers, removing duplicates, keeping only numbers >= 30,
  and returns the remaining numbers in descending order.

  Args:
    numbers: A list of numbers.

  Returns:
    A list of numbers that are >= 30, unique, and sorted in descending order.
  """
  unique_numbers = set(numbers)  # Remove duplicates efficiently
  filtered_numbers = [num for num in unique_numbers if num >= 30]  # Filter numbers >= 30
  filtered_numbers.sort(reverse=True)  # Sort in descending order
  return filtered_numbers
```

**3. Explanation of Changes:**

*   **Function Name:** Changed `process_nums` to `process_numbers` for better readability.
*   **Docstring:** Added a docstring to explain what the function does, its arguments, and its return value.  This is crucial for maintainability.
*   **Duplicate Removal:** Replaced the loop and `temp_list` with `unique_numbers = set(numbers)`.  This leverages the properties of a `set` to efficiently remove duplicates in O(n) time complexity on average, compared to the original code's O(n^2) complexity in the worst case.
*   **Filtering:** Replaced the `while` loop and `new_list` with a list comprehension: `filtered_numbers = [num for num in unique_numbers if num >= 30]`. This combines the filtering step into a single, more readable line.
*   **Sorting:** Combined the sorting and reversing steps into a single line: `filtered_numbers.sort(reverse=True)`.  This is more concise and efficient.
*   **Variable Names:** Changed `temp_list` to `unique_numbers` and `new_list` to `filtered_numbers` for increased clarity.

**Why these changes improve the original code:**

*   **Efficiency:** Using a `set` for duplicate removal significantly improves performance, especially for large lists.
*   **Readability:**  List comprehensions and more descriptive variable names make the code easier to understand.
*   **Conciseness:** Reducing the number of lines of code while maintaining clarity makes the code easier to maintain and reason about.
*   **Maintainability:** The docstring and improved structure make the code easier to understand and modify in the future.


## Next steps

Be sure to explore other examples of prompting in the repository. Try writing prompts for reviewing your own code as well using the example in this notebook.